In [ ]:
import os
import findspark
import sys

os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-21"
os.environ["SPARK_HOME"] = "C:\\Users\\AnMV\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\pyspark"

findspark.init()

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

# Tạo SparkConf và cấu hình các tham số
conf = SparkConf() \
    .setMaster('local') \
    .setAppName('Lab2.2 Temperature by Location') 

# Tạo SparkSession từ SparkConf
spark = SparkSession.builder.config(conf=conf).getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")

# Lấy SparkContext từ SparkSession
sc = spark.sparkContext

In [ ]:
DATA_IN = 'data_lab2.2_temperature_by_location.csv'

rdd = sc.textFile(DATA_IN)

# remove first line as header
headers = rdd.first()
rdd = rdd.filter(lambda x: x!=headers) 
rdd.collect()

In [ ]:
# Hàm chuyển đổi dữ liệu thành các cặp Key-Vaule tương ứng
def parseLine(line):
    station_id = line.split(',')[0]
    entry_type = line.split(',')[2]
    temperature = float(line.split(',')[3])*0.1*(9.0/5.0) + 32.0
    return (station_id, entry_type, temperature)

rdd_parsed = rdd.map(lambda x: parseLine(x))
rdd_parsed.collect()

In [ ]:
# Lọc các giá trị có Type là TMIN
min_temps = rdd_parsed.filter(lambda x: x[1] == 'TMIN')
min_temps.collect()

In [ ]:
# Tìm giá trị nhiệt độ nhỏ nhất theo các thành phố.
station_temps = min_temps.map(lambda x: (x[0], x[2]))
# station_temps.collect()
min_temps_by_location = station_temps.reduceByKey(lambda x,y : x if x < y else y)

In [ ]:
results = min_temps_by_location.collect()
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))